In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
cd ..

/


In [4]:
cd content/drive/My Drive/Enigma/data

/content/drive/My Drive/Enigma/data


In [0]:
import numpy as np
import pandas as pd
import ast

In [0]:
authors = pd.read_csv('authors.csv')
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [0]:
def stringtolist(str):
    return ast.literal_eval(str)
def len_list(author_id):
  return len(author_id)

In [0]:
def interest_set(interests):
    return list(set([y for x in interests for y in x]))

In [27]:
authors.head()

,id,affiliation,published-papers,citations,h-index,p-equal-a,p-unequal-a,interests
id,,,,,,,,
1101155,1101155,"Northeastern University, Boston, MA, USA",1,0,0,0.0000,0.0000,"['social network', 'social network-enabled EAP..."
1377286,1377286,"Department of Computer Science, University of ...",2,14,0,4.5000,1.8750,"['D scheme', 'split task', 'mixed criticality ..."
1613605,1613605,NaN,1,0,0,0.0000,0.0000,"['present value', 'expected value', 'Random Ra..."
629007,629007,NaN,1,0,0,0.0000,0.0000,"['ride comfort', 'different vehicle speed', 'v..."
1390253,1390253,"United Institute of Informatics Problems, Nati...",6,5,2,1.3333,0.3611,"['spindle head', 'decision support system', 'm..."


In [0]:
def process(author_id):
    listrey = ast.literal_eval(author_id)
    pps,cits,hidx,pea,puea,aff_authors,interests=0,0,0,0,0,0,0
    interests,affs = [],[]
    for i in listrey:
        i = int(i)
        pps += authors.loc[i]['published-papers']
        cits += authors.loc[i]['citations']
        hidx += authors.loc[i]['h-index']
        pea += authors.loc[i]['p-equal-a']
        puea += authors.loc[i]['p-unequal-a']
        interests.append(authors.loc[i]['interests'])
        if (pd.isna(authors.loc[i]['affiliation']))==False:
          affs.append(authors.loc[i]['affiliation'])
    aff_authors = len(affs)
    return [pps,cits,hidx,pea,puea,aff_authors,interests]

In [0]:
authors.index = authors.id
train['new_data'] = train['author_id'].apply(process)
new_data = pd.DataFrame(train['new_data'].values.tolist(), columns=['published_papers','authors_citations','h_index','pequala','punequala','aff_authors','interests'])
train = pd.concat((train,new_data),1)
train.drop('new_data',1,inplace=True)
train['author_id'] = train['author_id'].apply(stringtolist)
train['authors'] = train['author_id'].apply(len_list)

In [0]:
def no_of_interests(data):
  length = 0
  for i in data:
    length += len(i.split(','))
  return length

In [0]:
train['interests'] = train['interests'].apply(no_of_interests)

In [0]:
authors_interests = 0
interests_list = []
for i in list(authors['interests']):
  singlelist = ast.literal_eval(i)
  for j in singlelist:
    interests_list.append(j)
authors_interests = len(set(interests_list))

In [0]:
train['interests_pct'] = (train['interests']/authors_interests)*100

In [68]:
train.head()

,author_id,year,venue_id,citations,id,published_papers,authors_citations,h_index,pequala,punequala,aff_authors,interests,authors,interests_pct
0,"[302080, 254034, 1650574, 1688060, 356214, 144...",2011.0,15632,0,162253,23,12,7,1.5929,2.0285,2,81,9,0.003248
1,[471182],2010.0,15752,0,42448,1,0,0,0.0000,0.0000,1,10,1,0.000401
2,[1252574],2012.0,19098,0,226093,27,48,4,39.6667,43.7222,1,10,1,0.000401
3,"[1700521, 1634966, 1560119, 1588537, 1657765]",2012.0,20323,29,281211,38,299,9,151.1667,120.1194,2,50,5,0.002005
4,"[750555, 1125222]",2010.0,12539,0,8973,4,0,0,0.0000,0.0000,0,20,2,0.000802


In [0]:
train2 = train
train2.to_csv('train2.csv')

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR

y = train2['citations']
X = train2.drop(['author_id','id','citations','interests','year','venue_id'],1)

In [0]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X.iloc[:,:-1] = sc.fit_transform(X.iloc[:,:-1])

In [0]:
X_train , X_test , y_train , y_test = train_test_split(X,y,test_size=0.2)

In [82]:
lr = LinearRegression()
lr.fit(X_train,y_train)
y_pred = lr.predict(X_test)
print(r2_score(y_test,y_pred))
import xgboost

# Let's try XGboost algorithm to see if we can get better results
xgb = xgboost.XGBRegressor(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.75,
                           colsample_bytree=1, max_depth=7)
xgb.fit(X_train,y_train)
print(r2_score(y_test, xgb.predict(X_test)))

0.11367184196612101
[18:40:52] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


0.15408548030149916


In [89]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten

NN_model = Sequential()

# The Input Layer :
NN_model.add(Dense(128, kernel_initializer='normal',input_dim = X_train.shape[1], activation='relu'))

NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))

NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile the network :
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['accuracy'])
NN_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 128)               1152      
_________________________________________________________________
dense_11 (Dense)             (None, 256)               33024     
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 257       
Total params: 34,433
Trainable params: 34,433
Non-trainable params: 0
_________________________________________________________________


In [0]:
NN_model.fit(X_train, y_train, epochs=50, batch_size=256, validation_split = 0.2)

In [0]:
from keras.layers import Dense, Activation, LeakyReLU,Dropout
from keras.models import Sequential
from keras.layers import LeakyReLU
from keras import regularizers
# we still need to clear a graph though

model = Sequential()  # it is a feed-forward network without loops like in RNN
model.add(Dense(16, input_shape=(8,)))  # the first layer must specify the input shape (replacing placeholders)

model.add(Dense(4))

model.add(Dense(1))

from keras.callbacks import EarlyStopping, ModelCheckpoint
model.compile(
      loss='mean_squared_error',
    optimizer='adam',
    metrics=['accuracy']
)

model.fit(
    X_train, 
    y_train,
    batch_size=64, 
    epochs=20,
    validation_data=(X_test, y_test),
    verbose=1
)

In [92]:
from sklearn import linear_model
reg = linear_model.Ridge(alpha=.5)
reg.fit(X_train,y_train)
y_predreg = reg.predict(X_test)
r2_score(y_test,y_predreg)

0.11368115648873267

In [0]:
test['new_data'] = test['author_id'].apply(process)
new_data = pd.DataFrame(test['new_data'].values.tolist(), columns=['published_papers','authors_citations','h_index','pequala','punequala','aff_authors','interests'])
test = pd.concat((test,new_data),1)
test.drop('new_data',1,inplace=True)
test['author_id'] = test['author_id'].apply(stringtolist)
test['authors'] = test['author_id'].apply(len_list)

In [0]:
test['interests'] = test['interests'].apply(no_of_interests)
test['interests_pct'] = (test['interests']/authors_interests)*100

In [0]:
test2 = test
test2.to_csv('test2.csv')

In [99]:
X_train.head()

,published_papers,authors_citations,h_index,pequala,punequala,aff_authors,authors,interests_pct
147179,-0.539527,-0.282294,-0.508044,-0.256775,-0.261294,-1.282175,-0.53563,0.000802
62424,-0.493333,-0.274022,-0.359362,-0.246559,-0.243136,0.090878,-0.53563,0.000561
104590,-0.308555,-0.249208,-0.210679,-0.221870,-0.220309,0.090878,-0.53563,0.000682
150544,-0.516430,-0.284362,-0.656726,-0.259329,-0.262591,-1.282175,0.14258,0.001203
158220,0.130292,-0.133410,0.532732,-0.111196,-0.183042,1.463931,0.82079,0.001604


In [105]:
test.head()

,published_papers,authors_citations,h_index,pequala,punequala,aff_authors,authors,interests_pct
0,-0.562625,-0.284362,-0.656726,-0.259329,-0.262591,-0.595648,-1.21384,0.000321
1,-0.239263,-0.253344,-0.210679,-0.239918,-0.229797,0.090878,-0.53563,0.000802
2,-0.539527,-0.276090,-0.359362,-0.252518,-0.257692,-1.282175,-0.53563,0.000321
3,0.453654,0.160223,0.532732,0.069458,0.223184,0.090878,-0.53563,0.000802
4,0.153390,-0.195445,0.384050,-0.188583,-0.180504,0.777405,0.82079,0.001604


In [0]:
test = test.iloc[:,4:]
test.drop(['interests'],1,inplace = True)

In [0]:
test.iloc[:,:-1] = sc.transform(test.iloc[:,:-1])

In [0]:
sub2 = {'id':test2['id']}
citations = lr.predict(test)
sub2['citations'] = citations
sub2 = pd.DataFrame(sub2)
sub2.to_csv('sub2.csv',index=False)

In [0]:
subxgb2 = {'id':test2['id']}
citations = xgb.predict(test)
subxgb2['citations'] = citations
subxgb2 = pd.DataFrame(subxgb2)
subxgb2.to_csv('subxgb2.csv',index=False)

In [108]:
sub2.head()

,id,citations
0,77152,4.238303
1,253904,5.827881
2,46386,4.344722
3,19572,7.630582
4,182480,7.367564


In [109]:
subxgb2.head()

,id,citations
0,77152,2.022874
1,253904,6.787819
2,46386,4.016860
3,19572,8.665321
4,182480,7.739607
